In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# file_name = "/content/drive/My Drive/ML-Undergrad-Assignments-Projects/Multi-class Logistic Regression/iris_multiclass.csv"

In [ ]:
#Import required python module(s)
import numpy as np 

In [ ]:
#load data from file
data = np.genfromtxt('iris_multiclass.csv', delimiter=',',skip_header=True)
# data = np.genfromtxt(file_name, delimiter=',',skip_header=True)

#Distribute data into train and test sets
X_train = data[:120,[0,1,2,3]]
Y_train = data[:120,5]

X_test = data[-30:,[0,1,2,3]]
Y_test = data[-30:,5]
print(Y_train)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


In [ ]:
#Define the required Sigmoid function
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
#Define the Raw implementation function to set the parameters (theta)

def fit_implementation(X_train, Y_train, learning_rate=0.0005, max_iteration=1000, debug=False):
    #Adding a column of 1's so that the first element of each input is always 1
    #It would be multiplied with theta_0 later
    X_train= np.insert(X_train, 0, values=1, axis=1)
    no_attributes = X_train.shape[1]
    
    #Initialize model parameters theta
    theta = np.zeros((no_attributes,1))
    
    #Run number of iterations
    for icount in range(max_iteration):
        #delta is the quantity that will be added with theta during updating theta
        delta = np.zeros((no_attributes,1))
        totalLogLikelihood = 0
        #Check each data point
        for instance, actualOutput in zip(X_train,Y_train):
            instance=instance.reshape(no_attributes,1)
            dotResult = np.dot(theta.T, instance)
            
            predictedValue=sigmoid(dotResult).squeeze()
            #Calculate the derivative value for this data point
            derivativeValue = instance*(actualOutput-predictedValue)
            #Calculate the amount to be added with theta
            delta += learning_rate*derivativeValue

            logLikelihood = actualOutput*np.log(predictedValue)+(1-actualOutput)*np.log(1-predictedValue)
            totalLogLikelihood += logLikelihood
        theta = theta + delta
        
        #After each 100 iteration, print the status
        if icount%100==0 and debug==True:
            print(icount)
            print(totalLogLikelihood)
            print(theta)
            
    print(totalLogLikelihood)
    print(theta)
    
    return theta


def multciClassFitImplementation(X_train, Y_train):
    #Determine the list unique classes (unique target variable values) 
    #Changes required here
    (unique_labels_arr, _) = np.unique(Y_train, return_counts=True)

    #For each uniqueclass, determine the best classifier/parameter/theta which best separates the class with others
    #You can temporarily modify Y_train data to achieve the target and can call the fit_implementation function
    parameters = dict()
    #Changes required here
    for label in unique_labels_arr:
      ## https://www.kite.com/python/answers/how-to-replace-elements-in-a-numpy-array-if-a-condition-is-met-in-python#:~:text=Use%20numpy.,that%20do%20not%20with%20y%20.
      ## make THIS class label to be 1, others all to be 0
      y_temp_train = Y_train
      
      y_temp_train = np.where(y_temp_train != label, -1, y_temp_train) # make other than this_label become -1

      y_temp_train = np.where(y_temp_train == label, 1, y_temp_train) # make this_label become 1

      y_temp_train = np.where(y_temp_train == -1, 0, y_temp_train) # make -1 become 0 so, we have [1, 0] only

      theta = fit_implementation(X_train, y_temp_train) # fit using above function (temp. Y_train)
      parameters[label] = theta # assign dictionary[this_label_key] = current_theta
        
    return parameters
        
parameters = multciClassFitImplementation(X_train, Y_train) ## parameters = thetas

-1.3879260321845126
[[ 0.28590272]
 [ 0.46011454]
 [ 1.60972781]
 [-2.50791129]
 [-1.14796962]]
-64.76128269268942
[[ 0.63087103]
 [ 0.42606365]
 [-1.40136046]
 [ 0.32136084]
 [-0.77762565]]
-19.378481198390663
[[-0.93309689]
 [-1.69285985]
 [-1.54003974]
 [ 2.58282908]
 [ 2.00266044]]


In [ ]:
#One of the following parameters of the function is now thetas which is a dictionary containing (targetClass,theta) 
#as (key,value) pairs for all target classes
def prediction(X_test, Y_test, thetas):
    #Adding a column of 1's so that the first element of each input is always 1
    #It would be multiplied with theta_0 later
    X_test= np.insert(X_test, 0, values=1, axis=1)
    no_attributes = X_test.shape[1]
    
    correctCount = 0
    totalCount = 0
    
    maxPredictedValue = -10000
    predictedClass = 1.0
    
    #Check each data point
    for instance, actualOutput in zip(X_test,Y_test):
            instance=instance.reshape(no_attributes,1)
            #Determine the maximum predicted value and predictedClass
            #Changes required here
            
            ## h(theta, X) = theta.T * X
            maxPredictedValue = -10000
            predictedClass = 1.0
            for unique_label in thetas.keys():
              param_this_label = thetas[unique_label]
              predicted_val_this_label = param_this_label.T @ instance
              # print(f"predicted_val_this_label = {predicted_val_this_label}")
              if predicted_val_this_label >= maxPredictedValue:
                predictedClass = unique_label
                maxPredictedValue = predicted_val_this_label

            print(maxPredictedValue, predictedClass, actualOutput)
            if predictedClass == actualOutput:
                correctCount += 1
            totalCount += 1
    print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
    
prediction(X_test, Y_test, parameters)

[[3.46816928]] 1.0 1.0
[[4.5072006]] 1.0 1.0
[[3.8128889]] 1.0 1.0
[[4.68904186]] 1.0 1.0
[[4.1579075]] 1.0 1.0
[[-0.44931684]] 2.0 2.0
[[-0.84698976]] 2.0 2.0
[[-0.36527755]] 2.0 2.0
[[0.02562207]] 2.0 2.0
[[-0.21170313]] 2.0 2.0
[[1.14145311]] 3.0 3.0
[[1.27098693]] 3.0 3.0
[[0.87922695]] 3.0 3.0
[[1.88843296]] 3.0 3.0
[[1.23612787]] 3.0 3.0
[[4.61584723]] 1.0 1.0
[[2.45411658]] 1.0 1.0
[[3.97165712]] 1.0 1.0
[[3.51908296]] 1.0 1.0
[[3.52523329]] 1.0 1.0
[[-0.429165]] 2.0 2.0
[[-1.04322257]] 2.0 2.0
[[-0.36181706]] 2.0 2.0
[[-0.15370768]] 2.0 2.0
[[-0.77263985]] 2.0 2.0
[[2.22084681]] 3.0 3.0
[[0.39059426]] 3.0 3.0
[[2.06769182]] 3.0 3.0
[[2.47213954]] 3.0 3.0
[[2.37138782]] 3.0 3.0
Total Correct Count:  30  Total Wrong Count:  0  Accuracy:  100.0


# Expected Output: 
Total Correct Count:  30  Total Wrong Count:  0  Accuracy:  100.0